In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

from dask_gateway import Gateway
from dask.distributed import Client

gateway = Gateway()
cluster = gateway.new_cluster()

cluster.scale(10)
client = Client(cluster,timeout="120s")
cluster

In [2]:
from eofs.xarray import Eof
import xrscipy.signal as dsp
import math 

In [12]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

#model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2-WACCM','CESM2',\
#             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
#                 'FIO-ESM-2-0','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','KACE-1-0-G',\
#                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

model_names=['UKESM1-0-LL']

for x in range(0,len(model_names)):


    
    model_name='"'+model_names[x]+'"'
    experiment='"piControl"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15)).isel(time=slice(0,200*12))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    pcs = (pcs-pcs.mean('time'))/pcs.std('time')
    eofs=eofs.transpose("lat", "lon",'mode')
    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp585"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    pseudo_pcs = solver.projectField(sst_low, neofs=2)


    E_index=(pseudo_pcs.sel(mode=0)+pseudo_pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index_proj'+model_names[x]+'_ssp585_1.nc')
  

2023-03-24 20:23:38,788 - distributed.client - ERROR - Failed to reconnect to scheduler after 120.00 seconds, closing client
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/tornado/iostream.py", line 1389, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.10/ssl.py", line 1342, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLZeroReturnError: TLS/SSL connection has been closed (EOF) (_ssl.c:997)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 206, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.10/

In [4]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()



model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2-WACCM','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

for x in range(0,len(model_names)):

    def get_df(invar):
# this only needs to be created once
        gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
        zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
        mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
        outvar = xr.open_zarr(mapper, consolidated=True)

        return outvar
    
    model_name='"'+model_names[x]+'"'
    experiment='"piControl"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15)).isel(time=slice(0,200*12))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    pcs = (pcs-pcs.mean('time'))/pcs.std('time')
    eofs=eofs.transpose("lat", "lon",'mode')
    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp585"'
    
    def get_df(invar):
# this only needs to be created once
        gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
        zstore = invar.zstore.values[1]

# create a mutable-mapping-style interface to the store
        mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
        outvar = xr.open_zarr(mapper, consolidated=True)

        return outvar

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    pseudo_pcs = solver.projectField(sst_low, neofs=2)


    E_index=(pseudo_pcs.sel(mode=0)+pseudo_pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index_proj'+model_names[x]+'_ssp585_2.nc')
  

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:150: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out 

In [5]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()



model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2-WACCM','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

for x in range(0,len(model_names)):

    def get_df(invar):
# this only needs to be created once
        gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
        zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
        mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
        outvar = xr.open_zarr(mapper, consolidated=True)

        return outvar
    
    model_name='"'+model_names[x]+'"'
    experiment='"piControl"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15)).isel(time=slice(0,200*12))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    pcs = (pcs-pcs.mean('time'))/pcs.std('time')    
    eofs=eofs.transpose("lat", "lon",'mode')
    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp585"'
    
    def get_df(invar):
# this only needs to be created once
        gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
        zstore = invar.zstore.values[2]

# create a mutable-mapping-style interface to the store
        mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
        outvar = xr.open_zarr(mapper, consolidated=True)

        return outvar

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    pseudo_pcs = solver.projectField(sst_low, neofs=2)


    E_index=(pseudo_pcs.sel(mode=0)+pseudo_pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index_proj'+model_names[x]+'_ssp585_3.nc')
  

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:150: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out 

In [6]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2-WACCM','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"piControl"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid


    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15)).isel(time=slice(0,200*12))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = (pcs-pcs.mean('time'))/pcs.std('time')    
    pcs = solver.pcs(npcs=2)
    eofs=eofs.transpose("lat", "lon",'mode')

    E_index=(pcs.sel(mode=0)+pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index'+model_names[x]+'_piControl.nc')

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:150: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out 

In [7]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2-WACCM','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']


for x in range(0,len(model_names)):


    
    model_name='"'+model_names[x]+'"'
    experiment='"piControl"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15)).isel(time=slice(0,200*12))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    pcs = (pcs-pcs.mean('time'))/pcs.std('time')    
    eofs=eofs.transpose("lat", "lon",'mode')
    
    model_name='"'+model_names[x]+'"'
    experiment='"1pctCO2"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')


    sst_low=sst_low.transpose("time", "lon", "lat")

    pseudo_pcs = solver.projectField(sst_low, neofs=2)


    E_index=(pseudo_pcs.sel(mode=0)+pseudo_pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index_proj'+model_names[x]+'_1pct.nc')
  

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:150: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out 

In [8]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']

for x in range(0,len(model_names)):


    
    model_name='"'+model_names[x]+'"'
    experiment='"piControl"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15)).isel(time=slice(0,200*12))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    pcs = (pcs-pcs.mean('time'))/pcs.std('time')   
    eofs=eofs.transpose("lat", "lon",'mode')
    
    model_name='"'+model_names[x]+'"'
    experiment='"abrupt-4xCO2"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    pseudo_pcs = solver.projectField(sst_low, neofs=2)


    E_index=(pseudo_pcs.sel(mode=0)+pseudo_pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index_proj'+model_names[x]+'_4x.nc')
  

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:150: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out 

In [9]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[0]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp126"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")
    
    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    eofs=eofs.transpose("lat", "lon",'mode')

    E_index=(pcs.sel(mode=0)+pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index'+model_names[x]+'_ssp1_1.nc')

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [10]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[1]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp126"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    eofs=eofs.transpose("lat", "lon",'mode')

    E_index=(pcs.sel(mode=0)+pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index'+model_names[x]+'_ssp1_2.nc')

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [11]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

def get_df(invar):
# this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
    zstore = invar.zstore.values[2]

# create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
    outvar = xr.open_zarr(mapper, consolidated=True)

    return outvar

model_names=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5-CanOE','CanESM5','CESM2',\
             'CNRM-CM6-1','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','FGOALS-g3',\
                 'FIO-ESM-2-0','IPSL-CM6A-LR','KACE-1-0-G',\
                    'MIROC6','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']
for x in range(0,len(model_names)):

    
    model_name='"'+model_names[x]+'"'
    experiment='"ssp126"'

    test='source_id == '+ model_name +' & activity_id=="CMIP" & table_id == "Omon" & variable_id == "thetao" & experiment_id == '+experiment

 #   theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
 #   tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment+'& member_id == "r1i1p1f1"')
    theta = df.query('source_id =='+ model_name +' & table_id == "Amon" & variable_id == "ts" & experiment_id =='+ experiment)
   # tauu=df.query('source_id =='+ model_name +'& table_id == "Amon" & variable_id == "tauu" & experiment_id =='+ experiment)

    import xesmf as xe # I use xesmf to regrid from the ocean curvelinear to lat/lon grid

    theta_pi=get_df(theta)
    theta_trop=theta_pi.sel(lon=slice(140,280)).sel(lat=slice(-15,15))
    sst_low=theta_trop['ts'].assign_coords(time=range(0,len(theta_trop.time)))
    sst_low.load()
    sst_low= dsp.bandpass(sst_low,1/(10*12),1/(1.5*12),dim='time')

    sst_low=sst_low.transpose("time", "lon", "lat")

    solver = Eof(sst_low)

    eofs = solver.eofs(neofs=2)
    pcs = solver.pcs(npcs=2)
    eofs=eofs.transpose("lat", "lon",'mode')

    E_index=(pcs.sel(mode=0)+pcs.sel(mode=1))#**(-1/2)
    E_index=E_index/math.sqrt(2)

    E_index.to_netcdf('C_index'+model_names[x]+'_ssp1_3.nc')

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/codin